In [ ]:
import cv2
import numpy as np

# ------------------------------------
# Load grayscale image
# ------------------------------------
img = cv2.imread("img2.tif", cv2.IMREAD_GRAYSCALE)

# ------------------------------------
# Threshold to isolate bright melt pool
# (Use Otsu so you don't choose manually)
# ------------------------------------
_, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# ------------------------------------
# Morphological refinement
# ------------------------------------
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
th_close = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)  # fill holes
th_open = cv2.morphologyEx(th_close, cv2.MORPH_OPEN, kernel)  # remove noise

# ------------------------------------
# Extract melt pool coordinates
# ------------------------------------
coords = np.column_stack(np.where(th_open > 0))

# Get extreme points
topmost    = coords[np.argmin(coords[:, 0])]
bottommost = coords[np.argmax(coords[:, 0])]
leftmost   = coords[np.argmin(coords[:, 1])]
rightmost  = coords[np.argmax(coords[:, 1])]

# Compute width & height
width  = rightmost[1] - leftmost[1]
height = bottommost[0] - topmost[0]

# Compute centroid
centroid_x = int(np.mean(coords[:, 1]))
centroid_y = int(np.mean(coords[:, 0]))

# ------------------------------------
# Draw results on a color version
# ------------------------------------
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# Draw bounding box
cv2.rectangle(img_color,
              (leftmost[1], topmost[0]),
              (rightmost[1], bottommost[0]),
              (0, 0, 255), 2)

# Draw centroid
cv2.circle(img_color, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

# ------------------------------------
# Print measurements
# ------------------------------------
print("Melt pool width:", width)
print("Melt pool height:", height)
print("Centroid:", (centroid_x, centroid_y))

# ------------------------------------
# Show results
# ------------------------------------
cv2.imshow("Original", img)
cv2.imshow("Binary segmentation", th_open)
cv2.imshow("Measured melt pool", img_color)

cv2.waitKey(0)
cv2.destroyAllWindows()

Melt pool width: 244
Melt pool height: 452
Centroid: (213, 722)
